In [720]:
import pandas as pd
import numpy as np
import ast 
import json
import requests
from datetime import datetime
from datetime import date
import os
import sys
import time
import math
from thefuzz import fuzz
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import ast

In [111]:
# Set your API key here
API_KEY = 'Vz7nSide5OocT948BbImlJTkCKREvs6mrTBf2AL5'
# S3 bucket url base + s3Path (in get request) = the full path to the files
####### Streaming services API #######
# This is an example of how to use the streaming services API to get hourly emissions data.

# Use this API for continuous data streams and avoiding paging through results.

# api parameters for the streaming emissions/apportioned/hourly endpoint

for k in range(7,22):
    main_df = []
    if k<0:
        k_str = '199'+str(10+k)
    elif 0<=k<10:
        k_str = '200'+str(k)
    else:
        k_str = '20'+str(k)
    for j in range(1,13):
        for i in range(1,32):
            if (i<10) & (j<10):
                parameters = {
                    'api_key': API_KEY,
                    'beginDate': k_str+'-0'+str(j)+'-0'+str(i),
                    'endDate': k_str+'-0'+str(j)+'-0'+str(i),
                }
            elif (i<10) & (j>=10):
                parameters = {
                    'api_key': API_KEY,
                    'beginDate': k_str+'-'+str(j)+'-0'+str(i),
                    'endDate': k_str+'-'+str(j)+'-0'+str(i),
                }
            elif (i>=10) & (j<10):
                parameters = {'api_key': API_KEY,
                    'beginDate': k_str+'-0'+str(j)+'-'+str(i),
                    'endDate': k_str+'-0'+str(j)+'-'+str(i),
                }
            else:
                parameters = {'api_key': API_KEY,
                    'beginDate': k_str+'-'+str(j)+'-'+str(i),
                    'endDate': k_str+'-'+str(j)+'-'+str(i),
                }
    
            if ((j==2)&(i>=29))|((j==4)&(i>=31))|((j==6)&(i>=31))|((j==9)&(i>=31))|((j==11)&(i>=31)):
                continue
            
            
            date_format = "%Y-%m-%d"
            beginDate = datetime.strptime(parameters['beginDate'], date_format)
            endDate = datetime.strptime(parameters['endDate'], date_format)
            if (endDate - beginDate).days > 31:
                warnings.warn("The request you're making could be too large for the streaming services API and may result in a bad request. Please consider using the bulk data api endpoint instead. An example can be found in the bulk_data_demo.py file.")


            max_retries = 10
            retry_delay = 3  # seconds
            streamingUrl = "https://api.epa.gov/easey/streaming-services/emissions/apportioned/hourly"
            
            for attempt in range(max_retries):
                try:
                    streamingResponse = requests.get(streamingUrl, params=parameters)
                    json_data = streamingResponse.json()
            
                    # Check if the JSON is a list (what we expect from a successful response)
                    if (isinstance(json_data, list))&(len(json_data)>1): 
                        break  # Exit loop if good data received
                    else:
                        print(f"Unexpected format on attempt {attempt+1}, retrying...")
            
                except ValueError as e:
                    print(f"JSON decoding failed on attempt {attempt+1}: {e}")
            
                # If we haven't broken out, wait and retry
                time.sleep(retry_delay)
            else:
                print(f"Failed to get valid data after {max_retries} attempts")
                continue  # Skip this iteration in the main loop
            
            # Now you can safely use json_data
            streamingResponse_df = pd.DataFrame(json_data)
            # making get request using the emissions/apportioned/hourly endpoint
            
            

            print(f"Date: {parameters['beginDate']}")
            #print("JSON response preview:", streamingResponse.json()[:3])  # Just show a few entries
            print("Length of JSON response:", len(json_data))
            
            # printing the response error message if the response is not successful
            print("Status code: "+str(streamingResponse.status_code))
            if (int(streamingResponse.status_code) > 399):
                sys.exit("Error message: "+streamingResponse.json()['error']['message'])
            
            #print("Field Mappings: "+str(streamingResponse.headers['X-Field-Mappings']))
            ## the upper line is for printing the list of columns
            # collecting data as a data frame
            streamingResponse_df = pd.DataFrame(streamingResponse.json())
            df_filtered = streamingResponse_df.loc[streamingResponse_df["opTime"] == 1]
            df_filtered = df_filtered.drop(["associatedStacks", "so2Mass","so2MassMeasureFlg","so2Rate","so2RateMeasureFlg",\
                                           "co2Mass","co2MassMeasureFlg","co2Rate","co2RateMeasureFlg","noxMass",\
                                           "noxMassMeasureFlg","noxRate","noxRateMeasureFlg","heatInputMeasureFlg",\
                                           "so2ControlInfo","noxControlInfo","pmControlInfo","hgControlInfo","programCodeInfo"], axis=1)
            main_df.append(df_filtered)
    df_result = pd.concat(main_df, ignore_index=True)
    ### in case wanted to say the data in excel format, it might get to size of 10-11 meg for each year
    ### I don't think all of it can be saved in one excel file
    chunk_size = 1000000
    num_chunks = math.ceil(len(df_result) / chunk_size)
    
    for ii in range(num_chunks):
        start = ii * chunk_size
        end = start + chunk_size
        if end > len(df_result):
            end = len(df_result)
        df_chunk = df_result.iloc[start:end]
        save_path = "~/Documents/GitHub/AMR-project/hourly datasets/year"+str(k)+"-"+str(ii)+".xlsx"
        df_chunk.to_excel(save_path, index=False, engine="openpyxl")

Date: 2007-01-01
Length of JSON response: 91104
Status code: 200
Date: 2007-01-02
Length of JSON response: 91104
Status code: 200
Date: 2007-01-03
Length of JSON response: 91104
Status code: 200
Date: 2007-01-04
Length of JSON response: 91104
Status code: 200
Date: 2007-01-05
Length of JSON response: 91104
Status code: 200
Date: 2007-01-06
Length of JSON response: 91104
Status code: 200
Date: 2007-01-07
Length of JSON response: 91104
Status code: 200
Date: 2007-01-08
Length of JSON response: 91104
Status code: 200
Date: 2007-01-09
Length of JSON response: 91104
Status code: 200
Date: 2007-01-10
Length of JSON response: 91104
Status code: 200
Date: 2007-01-11
Length of JSON response: 91104
Status code: 200
Date: 2007-01-12
Length of JSON response: 91104
Status code: 200
Date: 2007-01-13
Length of JSON response: 91104
Status code: 200
Date: 2007-01-14
Length of JSON response: 91104
Status code: 200
Date: 2007-01-15
Length of JSON response: 91104
Status code: 200
Date: 2007-01-16
Length o

In [ ]:
API_KEY = 'Vz7nSide5OocT948BbImlJTkCKREvs6mrTBf2AL5'
parameters2 = {'api_key': API_KEY}
streamingUrl2 = "https://api.epa.gov/FACT/1.0/facilities"
streamingResponse2 = requests.get(streamingUrl2, params=parameters2)
json_data2 = streamingResponse2.json()
streamingResponse_df2 = pd.DataFrame(json_data2['data'])

In [1187]:
# region_dict = streamingResponse_df2['region']
monit_Plans_list = streamingResponse_df2['monitoringPlans']
ids = region_dict.apply(lambda x: x['id'][0])
streamingResponse_df2['region_id'] = ids
streamingResponse_df2['unitStackName'] = [[] for _ in range(len(streamingResponse_df2))]
streamingResponse_df2['dateforsteam'] = [[] for _ in range(len(streamingResponse_df2))]
# oris_region_id = streamingResponse_df2.loc[:,['orisCode','region_id']]
for i in range(len(ids)):
    monit_plant_i = monit_Plans_list[i]
    for k in range(len(monit_plant_i)):
        
        monit_Loc_i0 = monit_plant_i[k]['monitoringLocations']
        unitStackName_i = ''
        for j in range(len(monit_Loc_i0)):
            unitStackName_i = unitStackName_i + monit_Loc_i0[j]['name'] +','
        
        unitStackName_i = unitStackName_i.rstrip(',')
        streamingResponse_df2.loc[i,'unitStackName'].append(unitStackName_i)
        year = math.floor(monit_plant_i[0]['beginYearQuarter']/10)
        month = monit_plant_i[0]['beginYearQuarter']%10
        if month<10:
            date = f"{year}-0{month}-28"
        else:
            date = f"{year}-{month}-28"
        streamingResponse_df2.loc[i,'dateforsteam'].append(date)


oris_unitStackName = streamingResponse_df2.loc[:,['orisCode','unitStackName','dateforsteam']]

save_path = "~/Documents/GitHub/AMR-project/hourly datasets/oris_unitStackName.xlsx"
oris_unitStackName.to_excel(save_path, index=False, engine="openpyxl")


# test = streamingResponse_df2['monitoringPlans']
# test2 = test[0]
# test3 = test2[0]['monitoringLocations']
# test4 = ''
# for i in range(len(test3)):
#     test4 = test4 + test3[i]['name'] +','

# test4 = test4.rstrip(',')
# math.floor(test2[0]['endYearQuarter']/10)
# test2[0]['endYearQuarter']%10

In [1363]:
load_level_list
streamingUrl3

'https://api.epa.gov/FACT/1.0/monitoringPlan/880110/4/2023-02-28'

In [1365]:
max_retries = 5
retry_delay = 3  # seconds
API_KEY = 'Vz7nSide5OocT948BbImlJTkCKREvs6mrTBf2AL5'
parameters3 = {'api_key': API_KEY}


load_path = "~/Documents/GitHub/AMR-project/hourly datasets/oris_unitStackName.xlsx"
oris_unitStackName = pd.read_excel(load_path)

load_path = "~/Documents/GitHub/AMR-project/hourly datasets/maxSteamLoads.xlsx"
maxSteamLoad_all = pd.read_excel(load_path)

# date = '2023-01-01'
### this part should be active if starting from the first plant 
# maxSteamLoad_all = pd.DataFrame({
#     'CAMD_PLANT_ID':[],
#     'year':[],
#     'maxSteamLoad':[]
# })

plant_list = oris_unitStackName['orisCode'].unique()
plant_list = plant_list[plant_list>=4042]

for facility_id in plant_list:
    print(f"plant {facility_id}")
    unitStack_temp = oris_unitStackName.loc[oris_unitStackName['orisCode']==facility_id,'unitStackName'].reset_index()
     
    unitStack_Name = unitStack_temp.loc[0,'unitStackName']
    date_temp = oris_unitStackName.loc[oris_unitStackName['orisCode']==facility_id,'dateforsteam'].reset_index()
    date = date_temp.loc[0,'dateforsteam']

    success = 0

    ##### use this part when the loaded data is used #####
    date = ast.literal_eval(date)
    unitStack_Name = ast.literal_eval(unitStack_Name)
    ######################################################

    for k in range(len(date)):

        

        USN_k = unitStack_Name[k]
        Date_k = date[k]
    
        for attempt in range(max_retries):
            try:
                streamingUrl3 = f"https://api.epa.gov/FACT/1.0/monitoringPlan/{facility_id}/{USN_k}/{Date_k}"
                streamingResponse3 = requests.get(streamingUrl3, params=parameters3)
                json_data3 = streamingResponse3.json()
            
                data = json_data3['data']
                # Check if the JSON is a list (what we expect from a successful response)
                if (isinstance(data['monitoringLocations'], list))&(len(data['monitoringLocations'])>0):
                    success = 1
                    break  # Exit loop if good data received
                else:
                    print(f"Unexpected format on attempt {attempt+1}, retrying...")
            
            except ValueError as e:
                print(f"JSON decoding failed on attempt {attempt+1}: {e}")
                Date_k = str(int(Date_k[:4])+1)+Date_k[4:]
            
            # If we haven't broken out, wait and retry
            time.sleep(retry_delay)
        else:
            print(f"Failed to get valid data after {max_retries} attempts")
            # continue  # Skip this iteration in the main loop
            # break
        if success == 1:
            print("success")
            break ## no need to go over other unitstack_names






    # type(data['monitoringLocations'])
    streamingResponse_df3 = pd.DataFrame(data['monitoringLocations'])
    load_level_df = streamingResponse_df3['loadLevels']
    load_level_list = load_level_df[0]
    
    max_load_values = []
    endDateHour	= []
    for item in load_level_list:
        if 'maxLoadValue' in item:
            max_load_values.append(item['maxLoadValue'])
            endDateHour.append(item['endDateHour'])
        else:
            max_load_values.append(None)
            endDateHour.append(None)



    df_ranges = pd.DataFrame({
        'value': max_load_values,
        'end_date': endDateHour
    })
    
    # Parse dates and extract years
    df_ranges['end_year'] = pd.to_datetime(df_ranges['end_date'], errors='coerce').dt.year
    
    # Create start years (previous end year + 1)
    df_ranges['start_year'] = df_ranges['end_year'].shift(1).fillna(1995) + 1
    df_ranges.loc[0, 'start_year'] = 1995  # First range starts from 1995
    
    
    
    # Assign values to years
    years = list(range(1995, 2023))
    year_to_value = {}
    
    for year in years:
        # Find which range this year belongs to
        mask = (df_ranges['start_year'] <= year) & (year <= df_ranges['end_year'].fillna(2022))
        matching_rows = df_ranges[mask]
        
        if not matching_rows.empty:
            year_to_value[year] = matching_rows.iloc[0]['value']
        else:
            year_to_value[year] = None
    
    # Convert to lists if needed
    assigned_years = list(year_to_value.keys())
    assigned_values = list(year_to_value.values())
    
    year_and_maxsteamload = pd.DataFrame({
        'CAMD_PLANT_ID' : facility_id,
        'year':assigned_years,
        'maxSteamLoad':assigned_values
    })
    
    year_and_maxsteamload
    maxSteamLoad_all = pd.concat([maxSteamLoad_all,year_and_maxsteamload])

    # if facility_id>10:
    #     break


# maxSteamLoad_all = pd.DataFrame(maxSteamLoad_all)
save_path = "~/Documents/GitHub/AMR-project/hourly datasets/maxSteamLoads.xlsx"
maxSteamLoad_all.to_excel(save_path, index=False, engine="openpyxl")

plant 4042
success
plant 4048
success
plant 4050
success
plant 4054
JSON decoding failed on attempt 1: Expecting value: line 1 column 1 (char 0)


KeyboardInterrupt: 

In [1283]:
# test = oris_region_id.loc[oris_region_id['orisCode']==facility_id,'region_id'].reset_index()
# test.loc[0,'region_id']
# type(data['monitoringLocations'])
# streamingUrl3
# isinstance(data['monitoringLocations'], list)
# len(data['monitoringLocations'])>0
# data['monitoringLocations']
# test = streamingResponse_df2['monitoringPlans']
# test2 = test[0]
# test3 = test2[0]['monitoringLocations']
# test4 = ''
# for i in range(len(test3)):
#     test4 = test4 + test3[i]['name'] +','

# test4 = test4.rstrip(',')
# math.floor(test2[0]['endYearQuarter']/10)
# test2[0]['endYearQuarter']%10
# test4

# save_path = "~/Documents/GitHub/AMR-project/hourly datasets/maxSteamLoads.xlsx"
# maxSteamLoad_all.to_excel(save_path, index=False, engine="openpyxl")
# plant_list = oris_unitStackName['orisCode'].unique()
# type(oris_unitStackName)
save_path = "~/Documents/GitHub/AMR-project/hourly datasets/maxSteamLoads.xlsx"
maxSteamLoad_all.to_excel(save_path, index=False, engine="openpyxl")

## what do next?
I think one thing that would be good to do ist to add another column of capacity of each generator, and then we would only keep observations with $\frac{gen}{cap}$ between $85-95\%$, or we  can also have efficiency dispersion histogram the same as Omer's

/tmp/ipykernel_1365841/3768001295.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  stata_cw_1stmerge.loc[stata_cw_1stmerge['name_match']>50,"bad match"] =0


In [716]:
# stata_data = pd.read_stata('~/Documents/GitHub/AMR-project/hourly datasets/fossil_PlantCapacity_allyears.dta')
# stata_data = stata_data[stata_data['Namecap_plant']>=20]
# stata_data.loc[stata_data['PlantName']==""]
#(stata_cw_1stmerge["no plant ID"]==1)*(stata_cw_1stmerge["CAMD_STATE"]!=stata_cw_1stmerge["State"])
#float(crosswalk_data['EIA_PLANT_ID'])
# temp=stata_cw_1stmerge[stata_cw_1stmerge["bad match"].astype(int)==1]
# temp
#temp[temp["CAMD_PLANT_ID" ]==228]
#(stata_cw_1stmerge["State"]=="")|(stata_cw_1stmerge["State"].isna())
#stata_cw_1stmerge.loc[1,]\
#stata_cw_1stmerge['name_match']>50

In [1395]:
excel_data_merge[excel_data_merge['steam_ratio']>1.5]
excel_data[(excel_data['facilityId']==4042)&(excel_data['hour']==12)&(excel_data['date']=='1995-01-03')]
# excel_data["grossLoad"] = excel_data["opTime"]*excel_data["grossLoad"]
# excel_data["steamLoad"] = excel_data["steamLoad"].fillna(0)
# excel_data["steamLoadis"] = excel_data["steamLoad"]*excel_data["opTime"]
# plant_hourly = excel_data.groupby(['stateCode','facilityName','facilityId','date','hour'])[['grossLoad','heatInput','steamLoadis']]\
#     .sum().reset_index()
# plant_hourly = plant_hourly.rename(columns={'facilityId': 'CAMD_PLANT_ID'})
# plant_hourly['year'] = year

# excel_data_merge = pd.merge(plant_hourly, maxSteamLoad_all, on=['CAMD_PLANT_ID','year'], how='left')
# excel_data_merge['steam_ratio'] = excel_data_merge['steamLoadis']/excel_data_merge['maxSteamLoad']

# plant_hourly
# excel_data
# excel_data_merge[(excel_data_merge['grossLoad']==0)&(excel_data_merge['steam_ratio']>0)]

# filtered_df[filtered_df['efficiency']==0]

merged_df = pd.merge(excel_data_merge, stata_cw_merged, on='CAMD_PLANT_ID', how='left')
merged_df["load_from_steam"] = (merged_df["grossLoad"]==0)&(merged_df["steam_ratio"]>0)
merged_df["load_from_steam"] = merged_df["load_from_steam"].astype(int)


merged_df['use_steam_load'] = merged_df.groupby(['stateCode','facilityName','CAMD_PLANT_ID'])['load_from_steam'].transform('mean')
merged_df['use_steam_load'] = merged_df['use_steam_load']>0.7
merged_df['use_steam_load'] = merged_df['use_steam_load'].astype(int)
merged_df["grossLoad"] = merged_df["grossLoad"] + merged_df['use_steam_load']*merged_df['steam_ratio']*merged_df['Namecap_plant']

merged_df

,stateCode,facilityName,CAMD_PLANT_ID,date,hour,grossLoad,heatInput,steamLoadis,year,maxSteamLoad,...,Year,Enertype1,Summercap_plant,Wintercap_plant,Namecap_plant,name_match,bad match,weird match,load_from_steam,use_steam_load
0,AL,Charles R Lowman,56,1995-01-01,0,409.0,4301.0,0.0,1995,265.0,...,1995-01-01,Coal,550.169983,550.169983,538.000000,61.0,0,False,0,0
1,AL,Charles R Lowman,56,1995-01-01,0,409.0,4301.0,0.0,1995,265.0,...,1995-01-01,Coal,550.169983,550.169983,538.000000,100.0,0,False,0,0
2,AL,Charles R Lowman,56,1995-01-01,0,409.0,4301.0,0.0,1995,265.0,...,1995-01-01,Coal,550.169983,550.169983,538.000000,100.0,0,False,0,0
3,AL,Charles R Lowman,56,1995-01-01,1,390.0,4108.8,0.0,1995,265.0,...,1995-01-01,Coal,550.169983,550.169983,538.000000,61.0,0,False,0,0
4,AL,Charles R Lowman,56,1995-01-01,1,390.0,4108.8,0.0,1995,265.0,...,1995-01-01,Coal,550.169983,550.169983,538.000000,100.0,0,False,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1630469,WY,Wyodak,6101,1995-05-27,21,388.0,4860.7,0.0,1995,404.0,...,1995-01-01,Coal,335.000000,335.000000,362.070007,100.0,0,False,0,0
1630470,WY,Wyodak,6101,1995-05-27,22,390.0,4912.4,0.0,1995,404.0,...,1995-01-01,Coal,335.000000,335.000000,362.070007,100.0,0,False,0,0
1630471,WY,Wyodak,6101,1995-05-27,22,390.0,4912.4,0.0,1995,404.0,...,1995-01-01,Coal,335.000000,335.000000,362.070007,100.0,0,False,0,0
1630472,WY,Wyodak,6101,1995-05-27,23,389.0,4858.2,0.0,1995,404.0,...,1995-01-01,Coal,335.000000,335.000000,362.070007,100.0,0,False,0,0


In [ ]:
stata_data = pd.read_stata('~/Documents/GitHub/AMR-project/hourly datasets/fossil_PlantCapacity_allyears.dta')
stata_data = stata_data[stata_data['Namecap_plant']>=20]
load_path = "~/Documents/GitHub/AMR-project/hourly datasets/"+"epa_eia_crosswalk"+".xlsx"
crosswalk_data = pd.read_excel(load_path,sheet_name="epa_eia_crosswalk", index_col=0)

crosswalk_data.loc[crosswalk_data["CAMD_FUEL_TYPE"]=="Wood","CAMD_FUEL_TYPE"] = "Biofuels & waste"
crosswalk_data.loc[crosswalk_data["CAMD_FUEL_TYPE"]=="Diesel Oil","CAMD_FUEL_TYPE"] = "Petroleum"
crosswalk_data.loc[crosswalk_data["CAMD_FUEL_TYPE"]=="Diesel Oil","CAMD_FUEL_TYPE"] = "Petroleum"
crosswalk_data.loc[crosswalk_data["CAMD_FUEL_TYPE"]=="Pipeline Natural Gas","CAMD_FUEL_TYPE"] = "Natural Gas"
crosswalk_data.loc[crosswalk_data["CAMD_FUEL_TYPE"]=="Process Gas","CAMD_FUEL_TYPE"] = "Other Gas"
crosswalk_data.loc[crosswalk_data["CAMD_FUEL_TYPE"]=="Residual Oil","CAMD_FUEL_TYPE"] = "Petroleum"
crosswalk_data.loc[crosswalk_data["CAMD_FUEL_TYPE"]=="Tire Derived Fuel","CAMD_FUEL_TYPE"] = "Tire-derived fuels"

crosswalk_data["no plant ID"] = crosswalk_data['EIA_PLANT_ID'].isna().astype(int)
crosswalk_data.loc[crosswalk_data["no plant ID"]==1,'EIA_PLANT_ID'] = crosswalk_data.loc[crosswalk_data["no plant ID"]==1,'CAMD_PLANT_ID']
crosswalk_data = crosswalk_data.rename(columns={'EIA_PLANT_ID': 'PlantID'})

stata_cw_1stmerge = pd.merge(stata_data, crosswalk_data, on='PlantID', how='right')

stata_cw_1stmerge['name_match'] = stata_cw_1stmerge.apply(fuzzy_match, axis=1)
stata_cw_1stmerge.loc[stata_cw_1stmerge['PlantName']=="",'name_match'] = 100

stata_cw_1stmerge["bad match"] = (stata_cw_1stmerge["no plant ID"]==1)*((stata_cw_1stmerge["CAMD_STATE"]!=stata_cw_1stmerge["State"])&\
                                                                        ~((stata_cw_1stmerge["State"]=="")|(stata_cw_1stmerge["State"].isna())))+\
    (stata_cw_1stmerge["no plant ID"]==1)*(stata_cw_1stmerge["CAMD_STATE"]==stata_cw_1stmerge["State"])*\
    (stata_cw_1stmerge["CAMD_FUEL_TYPE"]!=stata_cw_1stmerge["Enertype1"])


stata_cw_1stmerge.loc[stata_cw_1stmerge['name_match']>50,"bad match"] =0
#stata_cw_1stmerge = stata_cw_1stmerge[stata_cw_1stmerge["bad match"]==0]
stata_cw_1stmerge = stata_cw_1stmerge.groupby(['PlantID','CAMD_PLANT_ID','name_match',"bad match","CAMD_STATE","State",\
                                              'CAMD_FACILITY_NAME','PlantName',"CAMD_FUEL_TYPE",'Enertype1'])["no plant ID"].mean().reset_index()

save_path_newcw= "~/Documents/GitHub/AMR-project/hourly datasets/cw_modified.xlsx"
stata_cw_1stmerge.to_excel(save_path_newcw, index=False, engine="openpyxl")

cw_cut_data = stata_cw_1stmerge[['CAMD_PLANT_ID','PlantID','name_match',"bad match"]]

load_path = "~/Documents/GitHub/AMR-project/hourly datasets/maxSteamLoads.xlsx"
maxSteamLoad_all = pd.read_excel(load_path)

In [1311]:
#cw_cut_data = cw_cut_data.rename(columns={'EIA_PLANT_ID': 'PlantID'})
for k in range(-5,-4):
    year = 2000+k
    stata_time = str(year)+"-01-01"
    stata_data_filtered = stata_data[stata_data["Year"]==pd.Timestamp(stata_time)]
    stata_cw_merged = pd.merge(stata_data_filtered, cw_cut_data, on='PlantID', how='left')
    stata_cw_merged["weird match"] = stata_cw_merged['CAMD_PLANT_ID'].isna()
    stata_cw_merged.loc[stata_cw_merged['CAMD_PLANT_ID'].isna(),'name_match'] = 50
    stata_cw_merged.loc[stata_cw_merged['CAMD_PLANT_ID'].isna(),'bad match'] = 0
    stata_cw_merged.loc[stata_cw_merged['CAMD_PLANT_ID'].isna(),'CAMD_PLANT_ID'] = \
        stata_cw_merged.loc[stata_cw_merged['CAMD_PLANT_ID'].isna(),'PlantID']
    df_merged =[]
    for j in range(0,1):
        try:
            excel_name = 'year'+str(k)+'-'+str(j)
            load_path = "~/Documents/GitHub/AMR-project/hourly datasets/"+excel_name+".xlsx"
            excel_data = pd.read_excel(load_path, index_col=0)
            

            
            

            
            
            excel_data["grossLoad"] = excel_data["opTime"]*excel_data["grossLoad"]
            excel_data["steamLoad"] = excel_data["steamLoad"].fillna(0)
            excel_data["steamLoadis"] = excel_data["steamLoad"]*excel_data["opTime"]
            plant_hourly = excel_data.groupby(['stateCode','facilityName','facilityId','date','hour'])[['grossLoad','heatInput','steamLoadis']]\
                .sum().reset_index()
            plant_hourly = plant_hourly.rename(columns={'facilityId': 'CAMD_PLANT_ID'})
            plant_hourly['year'] = year
            
            excel_data_merge = pd.merge(plant_hourly, maxSteamLoad_all, on=['CAMD_PLANT_ID','year'], how='left')
            excel_data_merge['steam_ratio'] = excel_data_merge['steamLoadis']/excel_data_merge['maxSteamLoad']

            
            merged_df = pd.merge(excel_data_merge, stata_cw_merged, on='CAMD_PLANT_ID', how='left')
            merged_df["load_from_steam"] = (merged_df["grossLoad"]==0)&(merged_df["steam_ratio"]>0)
            merged_df["load_from_steam"] = merged_df["load_from_steam"].astype(int)
            
            
            merged_df['use_steam_load'] = merged_df.groupby(['stateCode','facilityName','CAMD_PLANT_ID'])['load_from_steam'].transform('mean')
            merged_df['use_steam_load'] = merged_df['use_steam_load']>0.7
            merged_df['use_steam_load'] = merged_df['use_steam_load'].astype(int)
            merged_df["grossLoad"] = merged_df["grossLoad"] + merged_df['use_steam_load']*merged_df['steam_ratio']*merged_df['Namecap_plant']
            

            merged_df['PlantName'] = merged_df['PlantName'].fillna("")
            ## (merged_df["PlantName"] == merged_df["facilityName"])  | (merged_df["PlantName"] == "") some names are 
            ## the same but in very different order that makes fuzzy match hard too  
            ## merged_df['name_match'] = merged_df.apply(fuzzy_match, axis=1)
            merged_df["name_match"] =  merged_df['heatInput']>0
            # Convert Boolean to integer (1 for match, 0 for no match)
            merged_df["name_match"] = merged_df["name_match"].astype(int)
            merged_df['gen_to_capacity'] =merged_df['grossLoad'] / merged_df['Namecap_plant']
            # Define the conditions
            conditions = [
                merged_df['gen_to_capacity'] > 1.35,
                (merged_df['gen_to_capacity'] > 1.3) & (merged_df['gen_to_capacity'] <= 1.35),
                (merged_df['gen_to_capacity'] > 1.25) & (merged_df['gen_to_capacity'] <= 1.3),
                (merged_df['gen_to_capacity'] > 1.2) & (merged_df['gen_to_capacity'] <= 1.25),
                (merged_df['gen_to_capacity'] > 1.15) & (merged_df['gen_to_capacity'] <= 1.2),
                (merged_df['gen_to_capacity'] > 1.1) & (merged_df['gen_to_capacity'] <= 1.15),
                (merged_df['gen_to_capacity'] > 1.05) & (merged_df['gen_to_capacity'] <= 1.1),
                (merged_df['gen_to_capacity'] > 1.0) & (merged_df['gen_to_capacity'] <= 1.05),
                (merged_df['gen_to_capacity'] > 0.95) & (merged_df['gen_to_capacity'] <= 1.0),
                (merged_df['gen_to_capacity'] > 0.9) & (merged_df['gen_to_capacity'] <= 0.95),
                (merged_df['gen_to_capacity'] > 0.85) & (merged_df['gen_to_capacity'] <= 0.9),
                (merged_df['gen_to_capacity'] > 0.8) & (merged_df['gen_to_capacity'] <= 0.85),
                (merged_df['gen_to_capacity'] > 0.75) & (merged_df['gen_to_capacity'] <= 0.8),
                (merged_df['gen_to_capacity'] > 0.7) & (merged_df['gen_to_capacity'] <= 0.75),
                (merged_df['gen_to_capacity'] > 0.65) & (merged_df['gen_to_capacity'] <= 0.7),
                (merged_df['gen_to_capacity'] > 0.6) & (merged_df['gen_to_capacity'] <= 0.65),
                (merged_df['gen_to_capacity'] > 0.55) & (merged_df['gen_to_capacity'] <= 0.6),
                (merged_df['gen_to_capacity'] > 0.5) & (merged_df['gen_to_capacity'] <= 0.55),
                (merged_df['gen_to_capacity'] > 0.45) & (merged_df['gen_to_capacity'] <= 0.5),
                (merged_df['gen_to_capacity'] > 0.4) & (merged_df['gen_to_capacity'] <= 0.45),
                (merged_df['gen_to_capacity'] > 0.35) & (merged_df['gen_to_capacity'] <= 0.4),
                (merged_df['gen_to_capacity'] > 0.3) & (merged_df['gen_to_capacity'] <= 0.35),
                (merged_df['gen_to_capacity'] > 0.25) & (merged_df['gen_to_capacity'] <= 0.3),
                (merged_df['gen_to_capacity'] > 0.2) & (merged_df['gen_to_capacity'] <= 0.25),
                (merged_df['gen_to_capacity'] <= 0.2)
            ]

            # Define the corresponding values
            choices = ['gen/cap>135%','135%>gen/cap>130%','130%>gen/cap>125%', '125%>gen/cap>120%', '120%>gen/cap>115%','115%>gen/cap>110%',\
                       '110%>gen/cap>105%', '105%>gen/cap>100%', '100%>gen/cap>95%','95%>gen/cap>90%',\
                       '90%>gen/cap>85%', '85%>gen/cap>80%', '80%>gen/cap>75%','75%>gen/cap>70%',\
                       '70%>gen/cap>65%', '65%>gen/cap>60%', '60%>gen/cap>55%','55%>gen/cap>50%',\
                       '50%>gen/cap>45%', '45%>gen/cap>40%', '40%>gen/cap>35%','35%>gen/cap>30%',\
                      '30%>gen/cap>25%','25%>gen/cap>20%','20%>gen/cap']
            merged_df['gen/cap-cats'] = np.select(conditions, choices, default='unknown')


            filtered_df = merged_df[(merged_df["name_match"] == 1)]
            filtered_df['efficiency'] = 3.412*filtered_df.loc[:,'grossLoad']/filtered_df.loc[:,'heatInput']
            filtered_df = filtered_df[filtered_df['efficiency']<1]
            avg_gen_per_generator = filtered_df.groupby([\
                'stateCode','facilityName','PlantID','gen/cap-cats'])[['efficiency','Summercap_plant',\
                                            'Wintercap_plant','grossLoad','heatInput','weird match']].mean()
            # Then, compute the group sizes
            group_sizes = filtered_df.groupby([
                'stateCode', 'facilityName', 'PlantID', 'gen/cap-cats'
            ]).size().rename('count')

            # Combine them
            avg_gen_per_generator = avg_gen_per_generator.join(group_sizes).reset_index()

            if j==0:
                df_merged = avg_gen_per_generator
                df_merged["efficiency_mean"] = df_merged["efficiency"]
                df_merged = df_merged.drop(columns=["efficiency"])
            else:
                df_merged = pd.merge(df_merged, avg_gen_per_generator, on=["PlantID", "gen/cap-cats"],\
                                     how='left', suffixes=('', '_from_df2'))
                #Create a new column for merged efficiency: use df1's value if it exists, otherwise use df2's
                df_merged['efficiency_mean'] = df_merged['efficiency_mean'].combine_first(df_merged['efficiency'])
                df_merged['efficiency'] = df_merged['efficiency'].combine_first(df_merged['efficiency_mean'])
                df_merged["efficiency_mean"] = (df_merged["efficiency_mean"]*j\
                    +df_merged["efficiency"])/(j+1)
                df_merged = df_merged.drop(columns=["efficiency"])
                duplicate_cols = [col for col in df_merged.columns if col.endswith('_from_df2')]
                df_merged.drop(columns=duplicate_cols, inplace=True)
        except FileNotFoundError:
            print(f"file year{k}-{j} is not there")
            break
    save_path = "~/Documents/GitHub/AMR-project/hourly datasets/year"+str(k)+"final.xlsx"
    df_merged.to_excel(save_path, index=False, engine="openpyxl")
    

KeyError: 'steamLoad'

In [694]:
#df_merged = df_merged.drop(columns=["efficiency"])
#df_merged["efficiency_mean"] = (df_merged["efficiency_mean"]*j\
#    +df_merged["efficiency"])/(j+1)
#df_merged = df_merged.drop(columns=["efficiency"])
#duplicate_cols = [col for col in df_merged.columns if col.endswith('_from_df2')]
#df_merged.drop(columns=duplicate_cols, inplace=True)
#excel_data["check"] = excel_data["opTime"]*excel_data["grossLoad"]
#print(" response preview:", excel_data[20:21])

# sample_df = filtered_df.sample(frac=0.1)

# #plant_hourly.loc[plant_hourly["CAMD_PLANT_ID"]==6025]
# sample_df.to_excel("~/Documents/GitHub/AMR-project/hourly datasets/2021sample.xlsx", index=False, engine="openpyxl")
# sample_df.loc[filtered_df['gen_to_capacity']>1.35]

In [696]:
# len(crosswalk_data)
# test = crosswalk_data[crosswalk_data['MATCH_TYPE_GEN']=='CAMD Unmatched']
# filtered_df = filtered_df[filtered_df['efficiency']<1]
# temp['PlantID'].nunique()



In [700]:
# save_path = "~/Documents/GitHub/AMR-project/hourly datasets/year"+str(k)+"final.xlsx"
# df_merged.to_excel(save_path, index=False, engine="openpyxl")

In [698]:
# merged_df["name_match"] = (merged_df["name_match"] == 1) | (merged_df["PlantName"] == "")
# (merged_df[(merged_df["name_match"] == 0)])


In [296]:

#merged_df['PlantName'] = merged_df['PlantName'].fillna("")
def fuzzy_match(row):
    return fuzz.token_set_ratio(row['CAMD_FACILITY_NAME'], row['PlantName'])  # or whatever threshold you want
    ## set back to facilityName if it's going to be used with merged_df,
    ## for stata_cw_1stmerge CAMD_FACILITY_NAME should be used
# Apply to each row
#merged_df['name_match'] = merged_df.apply(fuzzy_match, axis=1)
#merged_df.loc[merged_df['PlantName']=="",'name_match'] = 100

In [702]:
#similarity = fuzz.ratio('Rivesville Power Station', 'FirstEnergy Rivesville')

#print(similarity)  # prints a number between 0 and 100

# merged_df.loc[merged_df['name_match']<35]

In [704]:
# plt.hist(stata_cw_1stmerge["bad match"].astype(int) , bins=3, color='skyblue', edgecolor='black')

In [706]:
# final_all_df.loc[3000,]
# 

In [708]:
# Optional: ensure gen/cap-cats is ordered properly if needed
# from pandas.api.types import CategoricalDtype
# cat_type = CategoricalDtype(categories=[...], ordered=True)
# filtered_df["gen/cap-cats"] = filtered_df["gen/cap-cats"].astype(cat_type)

# final_all=[]
# for ii in range(-5,22):
#     load_address = "~/Documents/GitHub/AMR-project/hourly datasets/year"+str(k)+"final.xlsx"
#     temp_final = pd.read_excel(load_address)
#     final_all.append(temp_final)


# final_all_df = pd.concat(final_all, ignore_index=True)

# final_all_df['mean_eff_lowest'] = final_all_df.groupby('PlantID')['efficiency_mean'].transform('min')
# final_all_df['mean_eff_avr'] = final_all_df.groupby('PlantID')['efficiency_mean'].transform('mean')
# #final_all_df['demean'] = final_all_df['efficiency_mean'] - final_all_df['mean_eff_lowest']
# categories_to_exclude = ['gen/cap>135%','135%>gen/cap>130%','130%>gen/cap>125%', '125%>gen/cap>120%', '120%>gen/cap>115%','115%>gen/cap>110%',\
#                        '110%>gen/cap>105%', '105%>gen/cap>100%']
# final_all_df_clean = final_all_df[~final_all_df['gen/cap-cats'].isin(categories_to_exclude)]

# category_order = [ 'gen/cap>135%','135%>gen/cap>130%','130%>gen/cap>125%', '125%>gen/cap>120%', '120%>gen/cap>115%','115%>gen/cap>110%',\
#                        '110%>gen/cap>105%', '105%>gen/cap>100%','100%>gen/cap>95%','95%>gen/cap>90%',\
#                        '90%>gen/cap>85%', '85%>gen/cap>80%', '80%>gen/cap>75%','75%>gen/cap>70%',\
#                        '70%>gen/cap>65%', '65%>gen/cap>60%', '60%>gen/cap>55%','55%>gen/cap>50%',\
#                        '50%>gen/cap>45%', '45%>gen/cap>40%', '40%>gen/cap>35%','35%>gen/cap>30%',\
#                       '30%>gen/cap>25%','25%>gen/cap>20%','20%>gen/cap']
# category_order.reverse()

In [710]:


# # Fit the OLS model
# model = smf.ols('efficiency_mean ~ mean_eff_avr + C(Q("gen/cap-cats"))+ Summercap_plant + Wintercap_plant'\
#                 , data=final_all_df).fit()

# # Get the summary frame
# summary_frame = model.summary2().tables[1]

# # Filter rows corresponding to gen/cap-cats dummy variables
# coef_df = summary_frame.filter(like='C(Q("gen/cap-cats"))', axis=0)

# # Reset index for plotting
# coef_df = coef_df.reset_index().rename(columns={'index': 'category'})
# coef_df['category'] = coef_df['category'].str.replace('C(Q("gen/cap-cats"))[T.','',regex=False)
# coef_df['category'] = coef_df['category'].str.replace(']','',regex=False)
# reference_category = pd.DataFrame({'category': ['100%>gen/cap>95%'], 'Coef.': [0]})
# coef_df = pd.concat([coef_df, reference_category], ignore_index=True)
# #coef_df['category'] = pd.Categorical(df['category'], categories=category_order, ordered=True)
# coef_df['coef_pos'] = coef_df['Coef.'] - min(coef_df['Coef.'])
# # Display the regression results
# #print(model.summary())
# # Create the bar plot
# plt.figure(figsize=(10, 6))
# sns.barplot(data=coef_df, x='category', y='coef_pos', \
#              order=category_order,errorbar=None, palette='viridis')

# # Customize labels and title
# plt.title('Average Efficiency by Generation-to-Capacity Category')
# plt.xlabel('gen/cap Category')
# plt.ylabel('Average Efficiency')
# plt.xticks(rotation=45)
# plt.tight_layout()

# # Show plot
# plt.show()

In [712]:
#plant_dummies = [col for col in merged_df.columns if col.startswith('PlantID_')]
#merged_df.drop(columns=plant_dummies, inplace=True)
# merged_df["PlantID"] = merged_df["CAMD_PLANT_ID"]
# merged_df = pd.get_dummies(merged_df, columns=['PlantID'], drop_first=True)

# plant_dummies = [col for col in merged_df.columns if col.startswith('PlantID_')]
# plant_dummies[1]

In [714]:
# coef_df
https://api.epa.gov/FACT/1.0/facilities?api_key=Vz7nSide5OocT948BbImlJTkCKREvs6mrTBf2AL5